In [21]:
from dotenv import load_dotenv

In [22]:
load_dotenv()

True

In [23]:
from langchain_teddynote import logging
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


# Runnable 
- 사용자 정의 체인을 묶어서 체인으로 만든다. 이 체인을 묶을 때, 모든 모듈은(prompt, llm, outputparser...) Runnable 프로토콜을 가지고있다.
- Runnable에는 prompt, llm, outputparser, retriever, tool....

# Runnable Passthrough
- 데이터를 효과적으로 전달하기 위한 방법. 데이터를 받아서 전달한다.
- 즉, key-value에 RunnablePassthrough를 입력한다. RunnablePassthrough에는 사용자의 입력이 전달된다.
- 그렇다면 사용자가 값만 전달할 수 있다.

In [24]:
from langchain_core.prompts import PromptTemplate
#from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama

In [25]:
prompt = PromptTemplate.from_template("{num}의 10배는?")
llm = ChatOllama(model="llama3:8b", temperature=0)

chain = prompt | llm

In [26]:
# dictionary형태. 이게 기본형태다!
chain.invoke({"num": 5})

AIMessage(content='😊\n\n5의 10배 (5 times 10) is... 50! 🎉', response_metadata={'model': 'llama3:8b', 'created_at': '2024-07-04T07:06:24.2351002Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 4358721400, 'load_duration': 3821235800, 'prompt_eval_count': 17, 'prompt_eval_duration': 40069000, 'eval_count': 23, 'eval_duration': 494380000}, id='run-78244bb8-393e-452f-b41f-a47a3dbe24e3-0')

In [27]:
#변수가 하나라면 값만 전달해도 된다.
chain.invoke(5)

AIMessage(content='😊\n\n5의 10배 (5 times 10) is... 50! 🎉', response_metadata={'model': 'llama3:8b', 'created_at': '2024-07-04T07:07:04.5041679Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 806721700, 'load_duration': 2676900, 'prompt_eval_duration': 322252000, 'eval_count': 23, 'eval_duration': 479731000}, id='run-ad21acfb-ef5d-4678-9fef-9fdc6adcafa0-0')

In [28]:
from langchain_core.runnables import RunnablePassthrough

In [29]:
RunnablePassthrough().invoke({"num":10})

{'num': 10}

In [34]:
# RunnablePassthrough에는 사용자의 입력이 전달된다. 사용자가 키값을 모두 기억하지 못하지만 넣어야할 입력값을 알고있다.
runnable_chain = {"num": RunnablePassthrough()} | prompt | llm | StrOutputParser()

runnable_chain.invoke(10)

'😊\n\n10의 10배는 100입니다. (10 × 10 = 100)'

In [ ]:
# query라고 생각하자! - RunnablePassthrough을 넣음으로써, 이것을 입력으로 받는것이다!
# chain = ({"context": retriever, "qeustion": RunnablePassthrough()}
#         | prompt
#         | llm
#         | StrOutputParser
#         )

In [35]:
RunnablePassthrough().invoke({"num":1})

{'num': 1}

# RunnalblePassthrough.assign()
- 입력값으로 들어온 값의 key/value쌍과 새롭게 할당된 key/value쌍을 합친다.

In [36]:
# invoke가 실행되고, new_num에 들어가 계산이 된다.
(RunnablePassthrough.assign(new_num=lambda x : x["num"] * 3)).invoke({"num": 1})

{'num': 1, 'new_num': 3}

# Runnable parallel
- 2개의 chain을 병렬적으로 처리하기 위한 방법이다.

In [1]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [2]:
model = ChatOllama(model="llama3:8b")

In [3]:
model

ChatOllama(model='llama3:8b')

In [4]:
chain1 = (
    PromptTemplate.from_template("{country}의 수도는 어디야?")
    | model
    | StrOutputParser()
)

chain2 = (
    PromptTemplate.from_template("{country}의 면적은 얼마야?")
    | model
    | StrOutputParser()
)

In [5]:
combined = RunnableParallel(capital=chain1, area=chain2)
# cpaital=chain1 체인을 실행해서 결과에 답변을 넣어주는 key값

In [6]:
chain1.invoke({"country": "대한민국"})

'😊\n\nThe capital of the Republic of Korea (South Korea) is Seoul (, Seoul). 🏙️'

In [7]:
chain2.invoke({"country": "미국"})

'😊\n\nThe United States of America has a total area of approximately 9,833,517 square kilometers (3,805,927 square miles). This includes:\n\n* Land area: about 9,695,000 square kilometers (3,756,000 square miles)\n* Water area: about 138,500 square kilometers (53,600 square miles)\n\nTo put that in perspective, the United States is the third-largest country by land area, after Canada and China. 🌎'

In [8]:
combined.invoke({"country": "대한민국"})

{'capital': '😊\n\nThe capital of the Republic of Korea (South Korea) is Seoul! 🏙️',
 'area': '😊\n\nThe Republic of Korea, also known as South Korea, has a total area of approximately 100,363 square kilometers (38,750 square miles). It is the seventh largest country in East Asia and the ninth largest in the world. 🌏'}

In [9]:
chain1 = (
    PromptTemplate.from_template("{country1}의 수도는 어디야?")
    | model
    | StrOutputParser()
)

chain2 = (
    PromptTemplate.from_template("{country2}의 면적은 얼마야?")
    | model
    | StrOutputParser()
)

In [10]:
combined = RunnableParallel(capital=chain1, area=chain2)
# cpaital=chain1 체인을 실행해서 결과에 답변을 넣어주는 key값

In [11]:
combined.invoke({"country1": "대한민국", "country2":"미국"})

{'capital': '😊\n\nThe capital of South Korea is Seoul (, also known as Hanguk in Korean). Specifically, the city is divided into two parts: North Seoul () and South Seoul (), with the Han River running through it.',
 'area': '😊\n\nThe total area of the United States is approximately 9,833,517 square kilometers (3,805,927 square miles). This includes:\n\n* Land area: 9,695,957 square kilometers (3,761,799 square miles)\n* Water area: 137,560 square kilometers (53,128 square miles)\n\nSo, the United States has a total of about 10 million square kilometers (4 million square miles) in land and water areas combined. 🌊🏞️'}

### Runnable Parallel 심화

In [39]:
from langchain_core.runnables import RunnableParallel

In [42]:
runnable = RunnableParallel(
    passed = RunnablePassthrough(),

    extra = RunnablePassthrough.assign(multi=lambda x: x["num"] * 3),

    modified = lambda x: x["num"] + 1    
)
runnable.invoke({"num":1})
# num이 x에 들어간다고 생각하자

{'passed': {'num': 1}, 'extra': {'num': 1, 'multi': 3}, 'modified': 2}

In [44]:
chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country}의 수도는?")
    | llm
)

chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country}의 면적은?")
    | llm
)

In [45]:
combined_chain = RunnableParallel(capital = chain1, area = chain2)
combined_chain.invoke("대한민국")

{'capital': AIMessage(content='😊\n\nThe area of South Korea (대한민국) is approximately 100,363 square kilometers (38,750 square miles).', response_metadata={'model': 'llama3:8b', 'created_at': '2024-07-04T09:25:17.2009469Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 4079295900, 'load_duration': 3475836800, 'prompt_eval_count': 17, 'prompt_eval_duration': 37531000, 'eval_count': 28, 'eval_duration': 563053000}, id='run-e6bf3d1b-c72a-430b-9831-e54ba5c539fa-0'),
 'area': '😊\n\nThe area of South Korea, also known as the Republic of Korea (대한민국), is approximately 100,363 square kilometers (38,750 square miles). 🇰🇷'}

# 배치처리 정렬

In [12]:
chain1 = (
    PromptTemplate.from_template("{country}의 수도는 어디야?")
    | model
    | StrOutputParser()
)

chain2 = (
    PromptTemplate.from_template("{country}의 면적은 얼마야?")
    | model
    | StrOutputParser()
)

In [13]:
combined = RunnableParallel(capital=chain1, area=chain2)

In [14]:
chain1.batch([{"country": "대한민국"}, {"country":"미국"}])

['😊\n\nThe capital of South Korea is Seoul (,).',
 '😊 The capital of the United States is Washington, D.C. (short for District of Columbia).']

In [15]:
chain2.batch([{"country": "대한민국"}, {"country":"미국"}])

['😊\n\nThe Republic of Korea (South Korea) has a total area of approximately 100,363 square kilometers (38,750 square miles). It is the 109th largest country in the world by land area.',
 '😊\n\nThe total area of the United States is approximately 9,833,517 square kilometers (3,805,927 square miles). This includes:\n\n* Land area: 9,695,700 square kilometers (3,764,100 square miles)\n* Water area: 137,817 square kilometers (53,227 square miles)\n\nNote that these figures are approximate and may vary slightly depending on the source.']

In [16]:
combined.batch([{"country": "대한민국"}, {"country":"미국"}])

[{'capital': "That's an easy one! 🇰🇷 The capital of South Korea (Republic of Korea) is Seoul (서울)! 🏙️",
  'area': '😊\n\nThe area of the Republic of Korea (대한민국) is approximately 100,363 square kilometers (38,750 sq mi).'},
 {'capital': "😊\n\nThe capital of the United States is Washington, D.C. (short for District of Columbia). It's located on the East Coast, along the Potomac River, and is home to many iconic landmarks such as the White House, Capitol Building, and National Mall. 🏛️",
  'area': '😊\n\nThe area of the United States is approximately 9,833,517 square kilometers (3,805,927 square miles). This makes it one of the largest countries in the world by land area.'}]

# RunnableLambda, itemgetter
- 사용자 정의 함수를 매핑해주는 역할을 하는 runnable 객체
- 즉, 사용자 정의 함수를 RunnableLambda로 감싸서 prompt 입력으로 넣어준다.

In [54]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
#from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from datetime import datetime

def get_today(a):
    #오늘 날짜 가져오기
    return datetime.today().strftime("%b-%d")
# 왜 함수에 사용하지도 않는 매개변수가 들어가있는가? RunnableLambda를 할떄는 무조건 하나 이상의 매개변수가 있어야한다 - 더미 매개변수

get_today(None)

'Jul-04'

In [58]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

prompt = PromptTemplate.from_template(
    "{today}가  생일인 {n}명을 나열하세요. 생년월일을 표기해 주세요."
)

llm = ChatOllama(model="llama3:8b", temperature=0)

In [61]:
chain = (
    {"today": RunnableLambda(get_today), "n": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [63]:
print(chain.invoke(3))

Here are three people born on July 4th:

1. **John Adams** (July 4, 1735) - 2nd President of the United States
2. **Thomas Jefferson** (July 4, 1743) - 3rd President of the United States and principal author of the Declaration of Independence
3. **Evelyn "Evie" Stevens** (July 4, 1988) - American professional road racing cyclist


In [64]:
def get_today(a):
    print(f"입력받은 변수 a의 값: {a}")
    #오늘 날짜 가져오기
    return datetime.today().strftime("%b-%d")
# 왜 함수에 사용하지도 않는 매개변수가 들어가있는가? RunnableLambda를 할떄는 무조건 하나 이상의 매개변수가 있어야한다 - 더미 매개변수

get_today(None)

입력받은 변수 a의 값: None


'Jul-04'

In [66]:
#매개변수 a에 사용자가 지정한 값이 들어간다.
chain = (
    {"today": RunnableLambda(get_today), "n": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
print(chain.invoke(3))

입력받은 변수 a의 값: 3
Here are three people born on July 4th:

1. **John Adams** (July 4, 1735) - 2nd President of the United States
2. **Thomas Jefferson** (July 4, 1743) - 3rd President of the United States and principal author of the Declaration of Independence
3. **Evelyn "Evie" Stevens** (July 4, 1988) - American professional road racing cyclist


In [67]:
print(chain.invoke({"n":3}))

입력받은 변수 a의 값: {'n': 3}
Here are three people born on July 4th, along with their birth dates:

1. **John Adams** - July 4, 1735
2. **Thomas Jefferson** - July 4, 1743
3. **Donald Trump** - July 4, 1946
